## Import Libraries

In [149]:
import numpy as np
import pandas as pd
import json
import nltk
import string
from sklearn.model_selection import train_test_split
import tensorflow as tf


from tensorflow.keras.datasets import imdb
import pandas as pd
import numpy as np
from tensorflow.keras.layers import LSTM, Activation, Dropout, Dense, Input
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Model
import string
import re
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import tensorflow.keras
from sklearn.model_selection import train_test_split

## Read data

In [115]:
data = open('Sarcasm_Headlines_Dataset.json', )

In [116]:
df = pd.DataFrame(columns = ['is_sarcastic', 'headline', 'article_link'])
for row in data:
    df = pd.concat([df, pd.json_normalize(json.loads(row))])

In [117]:
df = df.reset_index(drop=True)

In [118]:
df.head()

,is_sarcastic,headline,article_link
0,1,thirtysomething scientists unveil doomsday clo...,https://www.theonion.com/thirtysomething-scien...
1,0,dem rep. totally nails why congress is falling...,https://www.huffingtonpost.com/entry/donna-edw...
2,0,eat your veggies: 9 deliciously different recipes,https://www.huffingtonpost.com/entry/eat-your-...
3,1,inclement weather prevents liar from getting t...,https://local.theonion.com/inclement-weather-p...
4,1,mother comes pretty close to using word 'strea...,https://www.theonion.com/mother-comes-pretty-c...


In [119]:
data = df.copy()

### We can drop the link column for now

In [120]:
df.drop(['article_link'], axis = 1, inplace = True)

## Shape of the data

In [121]:
df.shape

(28619, 2)

## Check for the ratio os sarcastic headlines

In [126]:
df['is_sarcastic'].value_counts()

0    14985
1    13634
Name: is_sarcastic, dtype: int64

## Check for empty values

In [122]:
df.isnull().sum()

is_sarcastic    0
headline        0
dtype: int64

In [123]:
df.head()

,is_sarcastic,headline
0,1,thirtysomething scientists unveil doomsday clo...
1,0,dem rep. totally nails why congress is falling...
2,0,eat your veggies: 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...
4,1,mother comes pretty close to using word 'strea...


In [151]:
def remove_stopwords(data):
    data['headline without stopwords'] = data['headline'].apply(lambda x : ' '.join([word for word in x.split() if word not in (stopwords)]))
    return data

def remove_tags(string):
    result = re.sub('<.*?>','',string)
    return result
    
data_without_stopwords = remove_stopwords(df)
data_without_stopwords['clean_headline']= data_without_stopwords['headline without stopwords'].apply(lambda cw : remove_tags(cw))
data_without_stopwords['clean_headline'] = data_without_stopwords['clean_headline'].str.replace('[{}]'.format(string.punctuation), ' ')


In [152]:
data_without_stopwords.head()

,is_sarcastic,headline,headline without stopwords,clean_headline
0,1,thirtysomething scientists unveil doomsday clo...,thirtysomething scientists unveil doomsday clo...,thirtysomething scientists unveil doomsday clo...
1,0,dem rep totally nails why congress is falling ...,dem rep totally nails congress falling short g...,dem rep totally nails congress falling short g...
2,0,eat your veggies 9 deliciously different recipes,eat veggies 9 deliciously different recipes,eat veggies 9 deliciously different recipes
3,1,inclement weather prevents liar from getting t...,inclement weather prevents liar getting work,inclement weather prevents liar getting work
4,1,mother comes pretty close to using word stream...,mother comes pretty close using word streaming...,mother comes pretty close using word streaming...


Almost similar

## Find the length of each sentence after preprocessing

In [263]:
data_without_stopwords['Length'] = data_without_stopwords['clean_headline'].apply(lambda x : len(x.split(' ')))
max_length = data_without_stopwords['Length'].max()
max_length

106

### Pad sequences

In [264]:
vocabSize = 2000
tokenizer = Tokenizer(num_words=vocabSize, split=' ')
tokenizer.fit_on_texts(data_without_stopwords['clean_headline'].values)
X = tokenizer.texts_to_sequences(data_without_stopwords['clean_headline'].values)
X = pad_sequences(X)

### Train test split

In [265]:
from sklearn.model_selection import train_test_split

Y = pd.get_dummies(data_without_stopwords['is_sarcastic']).values
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.15, random_state = 42)
print(X_train.shape,Y_train.shape)
print(X_test.shape,Y_test.shape)

(24326, 35) (24326, 2)
(4293, 35) (4293, 2)


## Build model

In [270]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D, Bidirectional

embed_dim = 128
lstm_out = 196

model = Sequential()
model.add(Embedding(vocabSize, embed_dim,input_length = X.shape[1]))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(lstm_out, dropout=0.2, recurrent_dropout=0.2)))
model.add(Dense(2,activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
model.summary()

Model: "sequential_19"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_19 (Embedding)     (None, 35, 128)           256000    
_________________________________________________________________
spatial_dropout1d_14 (Spatia (None, 35, 128)           0         
_________________________________________________________________
bidirectional_12 (Bidirectio (None, 392)               509600    
_________________________________________________________________
dense_17 (Dense)             (None, 2)                 786       
Total params: 766,386
Trainable params: 766,386
Non-trainable params: 0
_________________________________________________________________


In [271]:
batch_size = 64
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 2)

Epoch 1/5
381/381 - 186s - loss: 0.5499 - accuracy: 0.7043
Epoch 2/5
381/381 - 126s - loss: 0.4439 - accuracy: 0.7847
Epoch 3/5
381/381 - 127s - loss: 0.4153 - accuracy: 0.8025
Epoch 4/5
381/381 - 365s - loss: 0.3936 - accuracy: 0.8111
Epoch 5/5
381/381 - 462s - loss: 0.3754 - accuracy: 0.8210


In [272]:
loss, acc = model.evaluate(X_test, Y_test)
print('Model loss is: ', loss)
print('Model accuracy is: ', acc)

135/135 [==============================] - 29s 213ms/step - loss: 0.4669 - accuracy: 0.7734
Model loss is:  0.46688002347946167
Model accuracy is:  0.7733519673347473


## Creating features and labels

In [273]:
headline = data_without_stopwords['clean_headline']

headline_list = []
for i in range(len(headline)):
    headline_list.append(headline[i])

sentiment = data_without_stopwords['is_sarcastic']

## Train test split to build a bidirectional model

In [274]:
y = np.array(list(sentiment))

X_train, X_test, Y_train, Y_test = train_test_split(headline_list, y, test_size=0.2, random_state = 45)

In [275]:
tokenizer = Tokenizer(num_words=5000)
tokenizer.fit_on_texts(X_train)

words_to_index = tokenizer.word_index

## Glove embeddings and indexing words

In [276]:
with open('glove.6B.300d.txt', 'r', encoding='UTF-8') as f:
    words = set()
    word_to_vec_map = {}
    for line in f:
        w_line = line.split()
        curr_word = w_line[0]
        word_to_vec_map[curr_word] = np.array(w_line[1:], dtype=np.float64)
maxLen = 150

In [277]:
vocab_len = len(words_to_index)
embed_vector_len = word_to_vec_map['test'].shape[0]

emb_matrix = np.zeros((vocab_len, embed_vector_len))

for word, index in words_to_index.items():
    embedding_vector = word_to_vec_map.get(word)
    if embedding_vector is not None:
        emb_matrix[index-1, :] = embedding_vector


In [278]:
X_train_indices = tokenizer.texts_to_sequences(X_train)

X_train_indices = pad_sequences(X_train_indices, maxlen=maxLen, padding='post')

## Bidirectional model

In [295]:
model = Sequential()
model.add(Input(150))
model.add(Embedding(input_dim=vocab_len, output_dim=embed_vector_len, input_length=maxLen, weights = [emb_matrix], trainable=False))
model.add(SpatialDropout1D(0.4))
model.add(Bidirectional(LSTM(128, return_sequences=True)))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential_21"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_21 (Embedding)     (None, 150, 300)          7951200   
_________________________________________________________________
spatial_dropout1d_16 (Spatia (None, 150, 300)          0         
_________________________________________________________________
bidirectional_14 (Bidirectio (None, 150, 256)          439296    
_________________________________________________________________
dense_19 (Dense)             (None, 150, 1)            257       
Total params: 8,390,753
Trainable params: 439,553
Non-trainable params: 7,951,200
_________________________________________________________________


In [296]:
model.fit(X_train_indices, Y_train, batch_size=64, epochs=6)

Epoch 1/6
358/358 [==============================] - 256s 715ms/step - loss: 0.6971 - accuracy: 0.5110
Epoch 2/6
358/358 [==============================] - 256s 716ms/step - loss: 0.6914 - accuracy: 0.5244
Epoch 3/6
358/358 [==============================] - 256s 714ms/step - loss: 0.6592 - accuracy: 0.6107
Epoch 4/6
358/358 [==============================] - 256s 715ms/step - loss: 0.6348 - accuracy: 0.6404
Epoch 5/6
358/358 [==============================] - 256s 716ms/step - loss: 0.5963 - accuracy: 0.6775
Epoch 6/6
358/358 [==============================] - 256s 716ms/step - loss: 0.5580 - accuracy: 0.7078


In [297]:
X_test_indices = tokenizer.texts_to_sequences(X_test)

X_test_indices = pad_sequences(X_test_indices, maxlen=maxLen, padding='post')

In [298]:
loss, acc = model.evaluate(X_test_indices, Y_test)
print('Model loss is: ', loss)
print('Model accuracy is: ', acc)

179/179 [==============================] - 78s 433ms/step - loss: 0.5503 - accuracy: 0.7137
Model loss is:  0.5503225922584534
Model accuracy is:  0.7136548757553101


In [302]:
preds = model.predict(X_test_indices)

In [301]:
n = np.random.randint(0,5000)

if preds[n].any() > 0.5:
    print('Sarcastic')
else: 
    print('Not Sarcastic')

Sarcastic
